In [2]:
%%capture

!pip install bs4
!pip install nbformat
!pip install pandas
!pip install geopandas
!pip install requests
!pip install --upgrade xlrd
!pip install openpyxl

In [3]:
%%capture

%run ./Tidy.ipynb

# Test d'un autre modèle: faire du clustering 

On veut ici utiliser la masse d'observation dont on dispose avec la prévalence pour toutes les drogues et toutes les classes d'âge.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.cluster import KMeans  # pour kmeans
import seaborn as sns  # pour scatterplots

X_vars = ['Prevalence (%)','Drug']

# Une première idée rejetée

Nous avons en premier lieu essayé de prédire la légalisation ou non du cannabis au sein d'un même pays à l'aide d'un SVM prenant en entrée la prévalence dans chaque pays.

Cette première approche n'a pas porté les fruits escomptés et ce pour plusieurs raisons, la principale étant une inadéquation entre notre jeu de données (trop faible) et le modèle choisi. Cette dernière section nous permet d'illustrer les raisons de cet échec et de tirer des conclusions sur la démarche à adopter lors du choix et de la mise en place de modèles.

### Sélection des données

On s'était restreint dans le cadre de la prédiction de la légalisation ou non du cannabis dans chaque pays, au pourcentage d'individus ayant déjà consommé du cannabis dans leur vie pour chaque pays.
On se rend alors vite compte que cette restriction nous donne un jeu de données très faible (1 observation par pays, soit seulement 29 observations). Ce qui va poser problème par la suite.

In [90]:
import pandas as pd

# On récupère la table de légalisation que l'on a nettoyé en amont

legal = legal_pays_clean

# On prend la prévalence sur la vie entière

df = pd.read_csv('https://raw.githubusercontent.com/refouch/Projet_Python_2A/refs/heads/main/Data/EUDA/edr2024-gps-current-table-1.csv')

dfcurrent = df[df['Substance'] == 'Cannabis']

dfcurrentadults = dfcurrent[dfcurrent['Age'] == 'All adults (15-64)']

dfcurrentadultslf = dfcurrentadults[dfcurrentadults['Recall period'] == 'Lifetime']

# On effectue un merge les deux tables
dfadultslf_legal = dfcurrentadultslf.join(legal, on = 'Country')

In [167]:
# Maintenant on tente d'entraîner le modèle parce que psartek.

from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Recodage des variables catégorielles

dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})
dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})

X = dfadultslf_legal[['Prevalence (%)','Males (%)','Females (%)']]
Y = dfadultslf_legal['Recreational_num']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

# Création du classifieur SVM avec le paramètre C=1
svm_classifier = svm.SVC(C=1, kernel='linear')

# Entraînement du modèle sur l'ensemble d'estimation
svm_classifier.fit(X_train, Y_train)

# Prédictions sur l'ensemble de test
Y_pred = svm_classifier.predict(X_test)

# Accuracy score = Normalization on correctly predicted outputs
accuracy_score = sklearn.metrics.accuracy_score(Y_test, Y_pred)
print(accuracy_score)

# Precision score = ratio tp / (tp + fp) -> ratio de vrais positifs contre faux positifs
precision_score = sklearn.metrics.precision_score(Y_test, Y_pred, average= 'micro')
print(precision_score)

#Recall 
recall_score = sklearn.metrics.recall_score(Y_test, Y_pred, average= 'micro')
print(recall_score)

#F1
f1_score = sklearn.metrics.f1_score(Y_test, Y_pred, average= 'micro')
print(f1_score)

0.5
0.5
0.5
0.5


/tmp/ipykernel_151499/185401596.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})
/tmp/ipykernel_151499/185401596.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 2, 'Legal' : 1, 'Illegal' : 0})


# Encore des test: avec la table de lelay


In [ ]:
import pandas as pd

# On récupère la table de légalisation que l'on a nettoyé en amont

legal = legal_pays_clean

# On prend la prévalence sur la vie entière

dfcurrent = test

# On effectue un merge les deux tables
dfadultslf_legal = test.join(legal, on = 'Country')
dfadultslf_legal = dfadultslf_legal.dropna()

,Country,Year of Treatment,All substances (global),Cannabis (global),All substances (deja traite.es),Cannabis (deja traite.es),All substances (jamais traite.es),Cannabis (jamais traite.es),Court / probation / police.global,General practitioner.global,...,Opioids.jamais,Cocaine.jamais,Stimulants.jamais,Hypnotics and Sedatives.jamais,Hallucinogens.jamais,Volatile Inhalants.jamais,Cannabis.jamais,Other substances.jamais,Recreational,Medical
2,Bulgaria,2022.0,1879.0,109.0,1164.0,42.0,605.0,66.0,28.0,1.0,...,84.0,37.0,105.0,18.0,2.0,0.0,66.0,25.0,Illegal,Illegal
8,Finland,2021.0,427.0,62.0,263.0,26.0,164.0,36.0,2.0,3.0,...,66.0,2.0,42.0,18.0,0.0,0.0,36.0,0.0,Illegal,Legal
26,Spain,2021.0,44347.0,12350.0,18359.0,3160.0,23482.0,8529.0,2005.0,1528.0,...,2463.0,10647.0,863.0,768.0,80.0,51.0,8529.0,81.0,Decriminalized,Legal


In [33]:
# Maintenant on tente d'entraîner le modèle parce que psartek.

from sklearn import svm
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Recodage des variables catégorielles

dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})
dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})

X = dfadultslf_legal[['Cannabis (global)','Cannabis (deja traite.es)','Cannabis (jamais traite.es)','Cannabis.jamais']]
Y = dfadultslf_legal['Recreational_num']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)

# Création du classifieur SVM avec le paramètre C=1
svm_classifier = svm.SVC(C=1, kernel='linear')

# Entraînement du modèle sur l'ensemble d'estimation
svm_classifier.fit(X_train, Y_train)

# Prédictions sur l'ensemble de test
Y_pred = svm_classifier.predict(X_test)

# Accuracy score = Normalization on correctly predicted outputs
accuracy_score = sklearn.metrics.accuracy_score(Y_test, Y_pred)
print(accuracy_score)

# Precision score = ratio tp / (tp + fp) -> ratio de vrais positifs contre faux positifs
precision_score = sklearn.metrics.precision_score(Y_test, Y_pred, average= 'micro')
print(precision_score)

#Recall 
recall_score = sklearn.metrics.recall_score(Y_test, Y_pred, average= 'micro')
print(recall_score)

#F1
f1_score = sklearn.metrics.f1_score(Y_test, Y_pred, average= 'micro')
print(f1_score)

1.0
1.0
1.0
1.0


/tmp/ipykernel_168213/4007374198.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Recreational_num'] = dfadultslf_legal['Recreational'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})
/tmp/ipykernel_168213/4007374198.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfadultslf_legal['Medical_num'] = dfadultslf_legal['Medical'].replace({'Decriminalized' : 1, 'Legal' : 1, 'Illegal' : 0})
